# 20 뉴스 그룹 분류

- News data(Text)를 분석하여 20개의 뉴스 그룹 중 어디에 속한 기사인지를 분류하는 모델

In [6]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

In [4]:
# 데이터 불러오기
news_data = fetch_20newsgroups(subset='all', random_state=156)

In [5]:
# 데이터의 key값을 확인
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [10]:
# target 클래스를 확인
print('target 클래스의 값과 분포도\n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들\n', news_data.target_names)

target 클래스의 값과 분포도
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [13]:
# 텍스트 데이터를 확인
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [16]:
# subset='train'으로 학습용 데이터만 추출, remove = ('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)

x_train = train_news.data
y_train = train_news.target

In [18]:
# subset='test'으로 테스트 데이터만 추출, remove = ('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)

x_test = test_news.data
y_test = test_news.target

In [20]:
print('학습 데이터의 크기 {0}, 테스트 데이터의 크기 {1}'.format(len(x_train), len(x_test)))

학습 데이터의 크기 11314, 테스트 데이터의 크기 7532


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(x_train)
x_train_cnt_vect = cnt_vect.transform(x_train)

In [22]:
# 학습 데이터로 fit()된 CountVectorizer를 이용하여 테스트 데이터를 피처 벡터화 변환 수행
x_test_cnt_vect = cnt_vect.transform(x_test)

In [23]:
print('학습 데이터 텍스트의 CountVectorizer Shape:', x_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(x_train_cnt_vect, y_train)
pred = lr_clf.predict(x_test_cnt_vect)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [28]:
print('CountVectorized Logistic Regression의 예측 정확도: {0:.3f}'.format(accuracy_score(pred, y_test)))

CountVectorized Logistic Regression의 예측 정확도: 0.608


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 텍스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(x_train)
x_train_tfidf_vect = tfidf_vect.transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

# LogisticRegression을 이용해 학습/예측/평가
lr_clf = LogisticRegression()
lr_clf.fit(x_train_tfidf_vect, y_train)
pred = lr_clf.predict(x_test_tfidf_vect)

In [31]:
print('TF-IDF Logistic Regrsssion의 예측 정확도: {0:3f}'.format(accuracy_score(pred, y_test)))

TF-IDF Logistic Regrsssion의 예측 정확도: 0.673659


In [32]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피쳐 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(x_train)
x_train_tfidf_vect = tfidf_vect.transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

lr_clf = LogisticRegression()
lr_clf.fit(x_train_tfidf_vect, y_train)
pred = lr_clf.predict(x_test_tfidf_vect)

In [33]:
print('TF-IDF Logistic Regrsssion의 예측 정확도: {0:3f}'.format(accuracy_score(pred, y_test)))

TF-IDF Logistic Regrsssion의 예측 정확도: 0.692246


In [34]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행, CV는 3 폴드 세트로 설정
params = {'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(x_train_tfidf_vect, y_train)

print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression best C parameter: {'C': 10}


In [36]:
# 최적의 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(x_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도: {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도: 0.701


In [38]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression객체를 lr_clf로 생성하는 Pipeline생성
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)), ('lr_clf', LogisticRegression(C=10))])

- Pipeline을 사용하면 별도의 TfidfVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요 없게 된다
- Pipeline의 fit()과 predict() 만으로 한번에 피처 벡터화와 ML학습/예측이 가능

In [39]:
# pipeline fit, predict
pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)

print('Pipeline을 통한 LogisticRegression의 예측 정확도: {0:.3f}'.format(accuracy_score(pred, y_test)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline을 통한 LogisticRegression의 예측 정확도: 0.701


In [ ]:
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')), ('lr_clf', LogisticRegression())])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range':[(1,1), (1,2), (1,3)],
          'tfidf_vect__max_df':[100, 300, 700],
          'lr_clf__C':[1, 5, 10]
          }

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)

grid_cv_pipe.fit(x_train, y_train)
grid_cv_pipe.best_params_
grid_cv_pipe.best_score_

pred = grid_cv_pipe.predict(x_test)
print('Pipeline을 통한 LogisticRegression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))